In [1]:
%load_ext autoreload
%autoreload 2

# ExpectationValue

In [2]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.opflow import PauliSumOp

observable = PauliSumOp.from_list([("XX", 1), ("YY", 2), ("ZZ", 3)])
print("observable\n", observable)

ansatz = RealAmplitudes(num_qubits=2, reps=2)
print("ansatz\n", ansatz)

observable
 1.0 * XX
+ 2.0 * YY
+ 3.0 * ZZ
ansatz
      ┌──────────┐     ┌──────────┐     ┌──────────┐
q_0: ┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├──■──┤ Ry(θ[4]) ├
     ├──────────┤┌─┴─┐├──────────┤┌─┴─┐├──────────┤
q_1: ┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[3]) ├┤ X ├┤ Ry(θ[5]) ├
     └──────────┘└───┘└──────────┘└───┘└──────────┘


In [3]:
from qiskit.quantum_info import Statevector

expval = Statevector(ansatz.bind_parameters([0, 1, 1, 2, 3, 5])).expectation_value(
    observable.primitive
)
print(expval.real)

1.8420921273153719


/Users/aa406165/Git/qiskit-core/qiskit/quantum_info/states/statevector.py:444: DeprecationWarning: The SparsePauliOp.table method is deprecated as of Qiskit Terra 0.19.0 and will be removed no sooner than 3 months after the releasedate. Use SparsePauliOp.paulis method instead.
  for z, x, coeff in zip(oper.table.Z, oper.table.X, oper.coeffs)


## ExpectationValue class

In [4]:
from qiskit.evaluators import PauliExpectationValue

### PauliExpectationValue

Evaluate the expectation value by sampling. This supports both AerSimulator and IBMQ backends.

In [5]:
from qiskit.providers.aer import AerSimulator
backend = AerSimulator()

In [6]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([0, 1, 1, 2, 3, 5], shots=3000)

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=1.872, variance=11.089767111111112, confidence_interval=[(1.7793853418573446, 1.9646146581426556)])

In [7]:
# pre-binding

circuit = ansatz.bind_parameters([0, 1, 1, 2, 3, 5])
expval = PauliExpectationValue(circuit, observable, backend=backend)
expval.evaluate()

no max_experiments for this backend: aer_simulator


ExpectationValueResult(value=1.85546875, variance=11.089408874511719, confidence_interval=[(1.6973055017137109, 2.0136319982862894)])

In [8]:
# multi prameters

expval = PauliExpectationValue(ansatz, observable, backend=backend)
expval.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8]], shots=3000)

no max_experiments for this backend: aer_simulator


CompositeResult(items=[ExpectationValueResult(value=1.8233333333333335, variance=11.134490222222222, confidence_interval=[(1.7305018616536993, 1.9161648050129676)]), ExpectationValueResult(value=0.14133333333333353, variance=10.466404444444445, confidence_interval=[(0.04752071653852258, 0.2351459501281445)])])

### Exact simulation by SaveExpectationValueVariance

In [9]:
from qiskit.evaluators import ExactExpectationValue

In [10]:
expval = ExactExpectationValue(ansatz, observable, backend=backend)
%time expval.evaluate([0, 1, 1, 2, 3, 5])

CPU times: user 6.93 ms, sys: 749 µs, total: 7.68 ms
Wall time: 7.14 ms


ExpectationValueResult(value=1.8420921273153708, variance=6.432763522806668, confidence_interval=None)

In [11]:
%time expval.evaluate([1, 1, 2, 3, 5, 8])

CPU times: user 2.44 ms, sys: 597 µs, total: 3.03 ms
Wall time: 2.63 ms


ExpectationValueResult(value=0.15611896641425527, variance=5.356814314603938, confidence_interval=None)

### Transpiled Circuits

In [12]:
from qiskit.test.mock import FakeBogota

backend = AerSimulator.from_backend(FakeBogota())

expval = PauliExpectationValue(ansatz, observable, backend=backend)
# expval.set_transpile_options(initial_layout=[1, 2])
for circ in expval.transpiled_circuits:
    print(circ)
    print(circ.metadata)
    print()
print(expval.evaluate([0, 1, 1, 2, 3, 5]))

global phase: π
               ┌────┐┌──────────────┐┌────┐┌────────┐     ┌────┐»
      q_0 -> 0 ┤ √X ├┤ Rz(θ[0] + π) ├┤ √X ├┤ Rz(3π) ├──■──┤ √X ├»
               ├────┤├──────────────┤├────┤├────────┤┌─┴─┐├────┤»
      q_1 -> 1 ┤ √X ├┤ Rz(θ[1] + π) ├┤ √X ├┤ Rz(3π) ├┤ X ├┤ √X ├»
               └────┘└──────────────┘└────┘└────────┘└───┘└────┘»
ancilla_0 -> 2 ─────────────────────────────────────────────────»
                                                                »
ancilla_1 -> 3 ─────────────────────────────────────────────────»
                                                                »
ancilla_2 -> 4 ─────────────────────────────────────────────────»
                                                                »
         c9: 2/═════════════════════════════════════════════════»
                                                                »
«               ┌──────────────┐┌────┐┌────────┐     ┌────┐┌──────────────┐»
«      q_0 -> 0 ┤ Rz(θ[2] + π) ├┤ √X ├┤ Rz(3π) ├─

### Transpile options and Run options

In [13]:
expval = PauliExpectationValue(ansatz, observable, backend=backend)
# setter
expval.set_transpile_options(optimization_level=2)
expval.set_run_options(shots=1_000_000)
expval.evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.5202442124039985, variance=12.0102925618336, confidence_interval=[(1.5148204914048522, 1.5256679334031449)])

In [14]:
# Method chain
expval.set_run_options(shots=8192).evaluate([0, 1, 1, 2, 3, 5])

ExpectationValueResult(value=1.564697265625, variance=11.957832276821136, confidence_interval=[(1.504947057782532, 1.624447473467468)])

In [15]:
# evaluate's option
expval.evaluate([0, 1, 1, 2, 3, 5], shots=300)

ExpectationValueResult(value=1.7, variance=11.834800000000001, confidence_interval=[(1.3898410777894128, 2.010158922210587)])

### Composite Evaluator

In [16]:
from qiskit.evaluators import JointEvaluator

In [17]:
len(expval.transpiled_circuits)

3

In [18]:
joint_evaluator = JointEvaluator([expval, expval, expval])

In [19]:
len(joint_evaluator.transpiled_circuits) # 3 × 3

9

In [20]:
joint_evaluator.evaluate([0, 1, 1, 2, 3, 5])

CompositeResult(items=[ExpectationValueResult(value=1.49609375, variance=12.035192966461182, confidence_interval=[(1.4360796528496154, 1.5561078471503846)]), ExpectationValueResult(value=1.57958984375, variance=11.982572674751282, confidence_interval=[(1.5197124510748754, 1.6394672364251246)]), ExpectationValueResult(value=1.530029296875, variance=11.991350591182709, confidence_interval=[(1.4701739004807295, 1.5898846932692705)])])

In [21]:
joint_evaluator.evaluate([[0, 1, 1, 2, 3, 5], [1, 1, 2, 3, 5, 8], [1, 2, 3, 5, 8, 13]])

CompositeResult(items=[ExpectationValueResult(value=1.498779296875, variance=12.051943480968475, confidence_interval=[(1.438702943477702, 1.558855650272298)]), ExpectationValueResult(value=0.180908203125, variance=11.619910180568695, confidence_interval=[(0.12106406448502109, 0.2407523417649789)]), ExpectationValueResult(value=0.962890625, variance=12.399816393852234, confidence_interval=[(0.9015053379061587, 1.0242759120938414)])])

### Mitigator

In [22]:
from qiskit.evaluators.backends import ReadoutErrorMitigation

backend = AerSimulator.from_backend(FakeBogota())
mit = ReadoutErrorMitigation(backend, mitigation="tensored", refresh=1, shots=2000, mit_pattern=[[0], [1]])
expval_raw = PauliExpectationValue(ansatz, observable, backend=backend)
expval_mit = PauliExpectationValue(ansatz, observable, backend=mit)
shots = 2000
print(f"w/o mitigation shots={shots}, result={expval_raw.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}")
print(f"w/  mitigation shots={shots}, result={expval_mit.evaluate([0, 1, 1, 2, 3, 5], shots=shots)}")

w/o mitigation shots=2000, result=ExpectationValueResult(value=1.5090000000000001, variance=11.951699, confidence_interval=[(1.3882004653590592, 1.629799534640941)])
w/  mitigation shots=2000, result=ExpectationValueResult(value=1.6670654250242893, variance=11.527989265818757, confidence_interval=[(1.5497912112226877, 1.784339638825891)])
